In [81]:
# import relevant libraries
import os
import jpegio as jio
import numpy as np
import subprocess
from PIL import Image


In [82]:
# Step 1: Initialize hyperparameter for k and n
sqrt_k = 8
k = 64
n = 70

# for traversing in zigzag order
zigzag = [
        [0, 1, 5, 6, 14, 15, 27, 28],
        [2, 4, 7, 13, 16, 26, 29, 42],
        [3, 8, 12, 17, 25, 30, 41, 43],
        [9, 11, 18, 24, 31, 40, 44, 53],
        [10, 19, 23, 32, 39, 45, 52, 54],
        [20, 22, 33, 38, 46, 51, 55, 60],
        [21, 34, 37, 47, 50, 56, 59, 61],
        [35, 36, 48, 49, 57, 58, 62, 63],
    ] 

In [83]:
# Step 2: Get the quantization table and DCT coefficient array
def get_qtable_ctable_from_jpg(path):
    image = jio.read(path)

    # print("Opened {img_path}, with size {height}x{width}".format(
    # img_path=path, height=image.image_height, width=image.image_width))
    
    # print("Shape of quantization table: " + str(qtable.shape))
    # print("Shape of coefficient table: " + str(ctable.shape))

    return np.copy(image.quant_tables[0]), np.copy(image.coef_arrays[0])

In [84]:
# Generates the histogram fro coefficient arrays
# Calculate the occurence from the same position in the block (1 to 64)
# Outputs a tuple (histogram, rmin), where rmin is the min value in histogram for comparison
def generate_hist_from_coeff(ctable):
    # Split into array of 8x8 blocks
    ctable_h, ctable_w = ctable.shape[:2]

    extra_h, extra_w = ctable_h % sqrt_k, ctable_w % sqrt_k
    # To make the height and width be multiple of sqrt_k for reshaping later
    if extra_h > 0:
        ctable = ctable[:-extra_h]
    
    if extra_w > 0:
        ctable = ctable[:, :-extra_w]

    num_h, num_w = ctable_h // sqrt_k, ctable_w // sqrt_k 
    num_blocks = (num_h) * (num_w)

    # transpose is to arrange the blocks into 2d matrix of blocks
    blocks = ctable.reshape(num_h, sqrt_k, num_w, sqrt_k).transpose(0, 2, 1, 3)
    # print("Shape of blocks: " + str(blocks.shape) + ", type: " + str(blocks.dtype))

    # counts[i] store the value from the i-th position in 8x8 block
    counts = np.ndarray((k, num_blocks), dtype=np.int32)

    for r in range(num_h):
        for c in range(num_w):
            for br in range(sqrt_k):
                for bc in range(sqrt_k):
                    counts[zigzag[br][bc], r * num_w + c] = blocks[r, c, br, bc]
    
    rmin = counts.min()
    rmax = counts.max()
    # print("Shape of counts: " + str(counts.shape))
    # print("The range of values is [{rmin}, {rmax}]".format(rmin=rmin, rmax=rmax))

    # the range of the value in histogram lies within [rmin, rmax]
    rlen = rmax - rmin + 1

    histograms = np.zeros((k, rlen), dtype=np.int32)
    for i in range(k):
        for j in range(counts.shape[1]):
            # shift by rmin
            histograms[i][counts[i][j] - rmin] += 1

    # print("Length of histograms: " + str(len(histograms)))
    # print(histograms[0])

    return histograms, rmin

In [85]:
# Save image into bmp format for compression purpose
def save_img_as_bmp(img_path, crop_len, output_img):
    with Image.open(img_path) as im:
        (left, upper, right, lower) = (crop_len, crop_len, im.width - crop_len, im.height - crop_len)
        im_crop = im.crop((left, upper, right, lower))

        im_crop.save(output_img)

# with Image.open("temp.bmp") as im_test:
#     print("The size of image is ({h}, {w})".format(h=im_test.height, w=im_test.width))

In [86]:
# Write the latest quantization table into qm.txt for compression later
def write_latest_qtable(qtable, output_file='qm.txt'):
    with open(output_file, "w") as f:
        for qrow in qtable:
            for ele in qrow:
                f.write(str(ele) + ' ')
            f.write('\n')

# Write constant matrices into qtable.txt to compress
def write_const_mat(i, output_file='qtable.txt'):
    with open(output_file, "w") as file:
        for _ in range(sqrt_k):
            file.write(' '.join([str(i)] * sqrt_k))
            file.write('\n')

In [87]:
# Compresses the given img to jpeg using the quantization table in qtable.txt
def compress_with_qtable(qtable_file, outfile, img):
    cjpeg = os.path.join(os.pardir, 'libjpeg-turbo', 'cjpeg')
    subprocess.run(cjpeg + ' -qtable {qtable} -outfile {outfile} {img}'.format(qtable=qtable_file, outfile=outfile, img=img))

# Compresses the given img to jpeg following the quality factor
def compress_with_quality(quality, outfile, img):
    cjpeg = os.path.join(os.pardir, 'libjpeg-turbo', 'cjpeg')
    subprocess.run(cjpeg + ' -quality {quality} -outfile {outfile} {img}'.format(quality=quality, outfile=outfile, img=img))

In [88]:
# Generate histograms for constant matrix with all elements equal i
def generate_hist_from_const_qtable(i):
    write_const_mat(i)

    const_qtable_file='qtable.txt'
    latest_qtable_file='qm.txt'
    first_compress_source='temp.bmp'
    first_compress_target='temp.jpg'
    second_compress_source='res.bmp'
    second_compress_target='res.jpg'

    # compress using the constant matrix
    compress_with_qtable(qtable_file=const_qtable_file, outfile=first_compress_target, img=first_compress_source)
    
    # save the temp.jpg as res.bmp to compress it again
    save_img_as_bmp(img_path=first_compress_target, crop_len=0, output_img=second_compress_source)

    # finally compress res.bmp to res.jpg using qm
    compress_with_qtable(qtable_file=latest_qtable_file, outfile=second_compress_target, img=second_compress_source)

    # get the coefficient array in res.jpg
    curr_qtable, curr_ctable = get_qtable_ctable_from_jpg(second_compress_target)

    return generate_hist_from_coeff(curr_ctable)

# Generate a list of histograms for different constant matrix 
# Outputs (histograms, mins) where histograms is the list of histogram and mins is the list of minimum value in each histogram
def generate_all_hist():
    histograms = []
    mins = []

    for i in range(1, n + 1):
        hist, rmin = generate_hist_from_const_qtable(i)
        histograms.append(hist)
        mins.append(rmin)
        # print("Finish process for x={i}".format(i=i))
    
    return histograms, mins
    

In [89]:
# Calculate the chi square distance between two histograms
def compare_two_histogram(hist1, rmin1, hist2, rmin2):
    rmax1, rmax2 = rmin1 + len(hist1) - 1, rmin2 + len(hist2) - 1

    # can split into three segments: 
    # 1: (min(rmin1, rmin2), max(rmin1, rmin2)) --> (l, ml)
    # 2: (max(rmin1, rmin2), min(rmax1, rmax2)) --> (ml, mr)
    # 3: (min(rmax1, rmax2), max(rmax1, rmax2)) --> (mr, r)
    l, r = min(rmin1, rmin2), max(rmax1, rmax2)

    chi_dist = 0

    for i in range(l, r + 1):
        # we use x to represent hist1, y to represent hist2
        # i not in [rmin1, rmax1] means it is out of bound, let it be 0
        x = 0 if i < rmin1 or i > rmax1 else hist1[i - rmin1]
        y = 0 if i < rmin2 or i > rmax2 else hist2[i - rmin2]
        
        if x + y == 0:
            continue

        chi_dist += ((x - y) ** 2) / (x + y)

    return chi_dist

In [90]:
# pos in range [0, 63]
def find_val_for_pos(i, D_jref, Dmin, hists, mins):
    chi_dists = []

    # hist = hists[idx], min = mins[idx]
    for idx, hist in enumerate(hists):
        chi_dists.append((idx + 1, compare_two_histogram(D_jref, Dmin, hist[i], mins[idx])))

    # sort chi_dists according to chi_dist computed
    chi_dists.sort(key=lambda x: x[1])

    return chi_dists[0][0]

# Find the second previous quantization table (the essence of the algorithm)
# img must be a jpeg extension (since we are extracting the quantization table)
def find_second_latest_qtable(dir, img):
    img_path = os.path.join(dir, img)
    
    # Get qtable and ctable from the image
    qtable, ctable = get_qtable_ctable_from_jpg(img_path)

    # Write the latest quantization table into qm.txt once for this current image
    write_latest_qtable(qtable, 'qm.txt')
    
    # Save cropped image as temp.bmp
    save_img_as_bmp(img_path, crop_len=4, output_img='temp.bmp')

    # Calculate the histogram for the reference histogram D_ref
    # The histogram for each constant qtable after compressing twice
    # Each histogram has length 64 that stores distribution of pos i in histogram[i]
    D_ref, Dmin = generate_hist_from_coeff(ctable=ctable)
    hists, mins = generate_all_hist()
    
    target_qtable = [1] * k

    for i in range(k):
        target_qtable[i] = find_val_for_pos(i, D_ref[i], Dmin, hists, mins)
    
    # print(target_qtable)
    # restore the target_qtable into 8x8 matrix
    res = [[1 for i in range(sqrt_k)] for j in range(sqrt_k)]

    for r in range(sqrt_k):
        for c in range(sqrt_k):
            # print(str(zigzag[r][c]) + " " + str(target_qtable[zigzag[r][c]]))
            res[r][c] = target_qtable[zigzag[r][c]]
    
    # write_latest_qtable(res, 'ans.txt')

    return res

In [91]:
def read_qtable(file):
    qtable = [[1 for i in range(sqrt_k)] for j in range(sqrt_k)]
    r = 0

    with open(file, 'r') as f:
        for line in f.readlines():
            row = line.split(' ')

            for c in range(sqrt_k):
                qtable[r][c] = int(row[c])
            r += 1

    return qtable

In [92]:
def compare_two_table(ref, res):
    diff = np.zeros((sqrt_k, sqrt_k), dtype=np.float32)

    for r in range(sqrt_k):
        for c in range(sqrt_k):
            diff[r][c] = round(abs(ref[r][c] - res[r][c]) / ref[r][c], 4)
    
    print('The mean percentage error is {e}'.format(e=diff.mean()))
    print('The standard deviation of percentage error is {std}'.format(std=np.std(diff)))
    return diff

# diff = compare_two_table(ref, res)


In [93]:
# For testing purpose, we pass in a jpg image then compress it once (to simulate multiple compression)
# For the second compression, we can use different quality factor (which coresponse to a qtable in jpeg format)
# Then, using our algorithm calculate the mean and standard deviation of percentage error
# where percentage error is defined as |res_i - ref_i| / ref_i
def testing(dir, img, quality):
    path = os.path.join(dir, img)

    ref, _ = get_qtable_ctable_from_jpg(path)

    # Save the img as bmp for compression
    save_img_as_bmp(path, 0, 'source.bmp')
    # Compress image into another jpg
    compress_with_quality(quality, 'source.jpg', 'source.bmp')
    
    res = find_second_latest_qtable('.', 'source.jpg')
    # print(ref)
    # print(res)
    print('Result for quality {q} for {img}'.format(q=quality, img=path))
    compare_two_table(ref, res)

In [94]:
# res = find_second_latest_qtable('images', 'test3.jpg')
# ref = read_qtable('ref.txt')
def repeated_testing(dir, img):
    for i in [50, 70, 90]:
        testing(dir, img, i)

# testing('images', 'compass.jpg', 85)

In [96]:
repeated_testing('images', 'compass.jpg')

Result for quality 50 for images\compass.jpg
The mean percentage error is 0.6181187629699707
The standard deviation of percentage error is 0.4277811348438263
Result for quality 70 for images\compass.jpg
The mean percentage error is 0.429673433303833
The standard deviation of percentage error is 0.3204420208930969
Result for quality 90 for images\compass.jpg
The mean percentage error is 0.02753281220793724
The standard deviation of percentage error is 0.0513429157435894


In [97]:
repeated_testing('images', 'test1.jpg')

Result for quality 50 for images\test1.jpg
The mean percentage error is 0.932117223739624
The standard deviation of percentage error is 0.9748184084892273
Result for quality 70 for images\test1.jpg
The mean percentage error is 0.5492078065872192
The standard deviation of percentage error is 0.5957048535346985
Result for quality 90 for images\test1.jpg
The mean percentage error is 0.21988436579704285
The standard deviation of percentage error is 0.31942275166511536
